## Ball Picking Challenge

### Part 1: Navigation

The aim is to program the robot for the first part of the challenge: follow the line from the beginning to the destination point. To do so, you need to reuse the abilities learnt in weeks 1-3; please feel free to reuse the code of those notebooks and exercises.

In [19]:
from packages import initialization
from packages import pioneer3dx as p3dx
p3dx.init()

Found ROS controller /pioneer3dx_13416_ip_172_31_40_152


In [20]:

import cv2
import numpy
# The first function must return True if an obstacle is detected in front of the robot, or False otherwise.
def is_obstacle_detected():
    rightSide, frontSide = getSonars()
    # move forward until an obstacle is detected at the front
    return frontSide<MAX_WALL_THRESHOLD
# The second function is the line following behavior as seen in previous notebooks during this week.
def follow_line():
    print('Following the line')
    width=150.0
    onTarget = False
    while not is_obstacle_detected():
        try:
            cx, cy = line_centroid(p3dx.image)
            err = cx-width/2
            linear = 0.2
            angular = -0.01*err
            move(linear, angular)
            p3dx.tilt(-0.47)
        except:
            onTarget = is_on_target()
            if onTarget:
                break;
            p3dx.tilt(-0.10)
    if not onTarget:
        print('Obstacle detected')
    return onTarget
# The next function was developed in the previous week: the robot turns until it is approximately parallel to the wall.
def getWall():
    print('Get the wall')
    leftSide, frontSide = getleftSonars()
    # move forward until an obstacle is detected at the front
    while frontSide>MAX_WALL_THRESHOLD: 
        move(DEF_X_SPEED,0)
        leftSide, frontSide = getleftSonars()

    # turn until the wall is detected by sonar 0
    while p3dx.distance[0]>MAX_WALL_THRESHOLD:
        if frontSide<leftSide:
            yawSpeed = -DEF_YAW_SPEED * 3
        else:
            yawSpeed = -DEF_YAW_SPEED
        move(0,yawSpeed)
        leftSide, frontSide = getleftSonars()
    move(0,0)
# The next function is checked during the wall following behavior: it must return True when the line is again detected, or False otherwise.
def is_line_detected():
    hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
    lower_cyan = numpy.array([130,100,0])
    upper_cyan = numpy.array([180, 255, 255])
    mask = cv2.inRange(hsv, lower_cyan, upper_cyan)
    for col in range(0,150):
        for row in range(80,100):
            if mask[row,col]==255:
                return True
# Next, we reuse the wall following behavior from previous week.
def follow_wall():
    print('Following the wall')
    while not is_line_detected():
            leftSide, frontSide = getleftSonars()
            # by default, just move forward
            xSpeed = DEF_X_SPEED
            yawSpeed = 0
            # if we're getting too close to the wall with the front side...
            if frontSide<MIN_WALL_THRESHOLD:
                # go backward and turn right quickly (x4)
                xSpeed = -0.1
                yawSpeed = -DEF_YAW_SPEED * 4
            else:
                # if we're getting too close to the wall with the right side...
                if leftSide<MIN_WALL_THRESHOLD:
                    # move slowly forward (x0.5) and turn right
                    xSpeed  = DEF_X_SPEED*0.5
                    yawSpeed = -DEF_YAW_SPEED
                else:
                    # if we're getting too far away from the wall with the right side...
                    if leftSide>MAX_WALL_THRESHOLD:
                        # move slowly forward (x0.5) and turn right
                        xSpeed  = DEF_X_SPEED*0.5
                        yawSpeed = +DEF_YAW_SPEED
            # Move the robot
            move(xSpeed,yawSpeed)
    print('Line detected')
# Finally, a function is needed for turning the robot slightly until it is approximately parallel to the line again.
def getLine():
    print('Get line')
    lower_cyan = numpy.array([130,100,0])
    upper_cyan = numpy.array([180, 255, 255])
    exit = True
    while exit:
        linear = 0.05
        angular = -0.30
        move(linear, angular)
        hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
        mask = cv2.inRange(hsv, lower_cyan, upper_cyan)
        mask[0:90, 0:150] = 0
        for col in range(0,150):
            for row in range(91,100):
                if mask[row,col]==255:
                    exit = False
# Lower-level functions
DEF_X_SPEED = 0.2       # default forward velocity
DEF_YAW_SPEED = 0.15    # default turning velocity
MIN_WALL_THRESHOLD = 0.2
MAX_WALL_THRESHOLD = 0.3

def getSonars():
    ritghSide = min(p3dx.distance[5:8])
    frontSide = min(p3dx.distance[3:5])
    return ritghSide, frontSide
def getleftSonars():
    leftSide = min(p3dx.distance[0:3])
    frontSide = min(p3dx.distance[3:5])
    return leftSide, frontSide

def move(V_robot,w_robot):
    r = 0.1953 / 2
    L = 0.33
    w_r = (2 * V_robot + L * w_robot) / (2*r)
    w_l = (2 * V_robot - L * w_robot) / (2*r)
    p3dx.move(w_l, w_r)
    
def target_blob():
    # colore green 60 255 255
    lower_green = numpy.array([50,100,0])
    upper_green = numpy.array([70, 255, 255])
    hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv, lower_green, upper_green)
    M = cv2.moments(mask)
    area = M['m00']
    if area > 0:
        cx = int(M['m10']/area)
        cy = int(M['m01']/area)
    else:
        cx = 0
        cy = 0
    return cx,cy
    
def is_on_target():
    p3dx.tilt(-0.47)
    cx,cy = target_blob()
    if cx >= 70 and cx < 80 and cy >= 60:
        print("On Target")
        move(0,0)
        return True
    else:
        return False

def close_target():
    p3dx.tilt(-0.47)
    cx,cy = target_blob()
    if cx >= 70 and cx < 80 and cy >= 80:
        return True
    else:
        return False

def forward():
    print('dentro forward')
    target = 0.4        # target distance
    radius = 195.3/2000.0      # wheel radius
    initialEncoder = p3dx.leftEncoder
    print('leftEncoder forward', initialEncoder)
    distance = 0.0
    while distance < target:
        p3dx.move(1.0,1.0)
        angle = p3dx.leftEncoder - initialEncoder
        distance = angle*radius
    print('distanza', distance)
    p3dx.stop()
    
def line_centroid(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    lower_cyan = numpy.array([130,100,0])
    upper_cyan = numpy.array([180, 255, 255])
    mask = cv2.inRange(hsv, lower_cyan, upper_cyan)
    mask[0:80, 0:150] = 0
    M = cv2.moments(mask)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    return cx, cy
# The main loop looks very similar to the proposed algorithm:
p3dx.tilt(-0.47)
try:
    while True:
        if follow_line():
            move(0,0)
            break;
        getWall()
        follow_wall()
        getLine()
    while not close_target():
        move(0.1,0.0)
    move(0,0)
    forward()
except KeyboardInterrupt:
    move(0,0)


Following the line
Obstacle detected
Get the wall
Following the wall
Line detected
Get line
Following the line
On Target
dentro forward
('leftEncoder forward', 190.43232152339823)
('distanza', 0.4030969039367036)


---
#### Try-a-Bot: an open source guide for robot programming
Developed by:
[![Robotic Intelligence Lab @ UJI](img/logo/robinlab.png "Robotic Intelligence Lab @ UJI")](http://robinlab.uji.es)

Sponsored by:
<table>
<tr>
<td style="border:1px solid #ffffff ;">
<a href="http://www.ieee-ras.org"><img src="img/logo/ras.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="http://www.cyberbotics.com"><img src="img/logo/cyberbotics.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="http://www.theconstructsim.com"><img src="img/logo/theconstruct.png"></a>
</td>
</tr>
</table>

Follow us:
<table>
<tr>
<td style="border:1px solid #ffffff ;">
<a href="https://www.facebook.com/RobotProgrammingNetwork"><img src="img/logo/facebook.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="https://www.youtube.com/user/robotprogrammingnet"><img src="img/logo/youtube.png"></a>
</td>
</tr>
</table>